# Uncertainty-aware Deep Learning with SNGP

**Learning Objectives**:

* Understand what an distance-aware model is 
* Learn how to implement a deep residual model using the Keras Model API
* Learn how to implement a SNGP model on a deep classifier to improve its distance awareness.
* Learn how to train and evaluate a SNGP model 
* Understand the difference with other methods such as Monte Carlo dropout and deep ensemble models

In AI applications that are safety-critical, such as medical decision making and autonomous driving, or where the data is inherently noisy (for example, natural language understanding), it is important for a deep classifier to reliably quantify its uncertainty. The deep classifier should be able to be aware of its own limitations and when it should hand control over to the human experts. This tutorial shows how to improve a deep classifier's ability in quantifying uncertainty using a technique called **[Spectral-normalized Neural Gaussian Process (SNGP)](https://arxiv.org/abs/2006.10108)**.

The core idea of SNGP is to improve a deep classifier's _**distance awareness**_ by applying simple modifications to the network. A model's _distance awareness_ is a measure of how its predictive probability reflects the distance between the test example and the training data. This is a desirable property that is common for gold-standard probabilistic models (for example, the [Gaussian process](https://en.wikipedia.org/wiki/Gaussian_process) with RBF kernels) but is lacking in models with deep neural networks. SNGP provides a simple way to inject this Gaussian-process behavior into a deep classifier while maintaining its predictive accuracy.

This notebooks implements a deep residual network (ResNet)-based SNGP model on [scikit-learn’s two moons](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html) dataset, and compares its uncertainty surface with that of two other popular uncertainty approaches: [Monte Carlo dropout](https://arxiv.org/abs/1506.02142) and [Deep ensemble](https://arxiv.org/abs/1612.01474).

This notebook illustrates the SNGP model on a toy 2D dataset. For an example of applying SNGP to a real-world natural language understanding task using a BERT-base, check out the [SNGP-BERT tutorial](https://www.tensorflow.org/text/tutorials/uncertainty_quantification_with_sngp_bert). For high-quality implementations of an SNGP model (and many other uncertainty methods) on a wide variety of benchmark datasets (such as [CIFAR-100](https://www.tensorflow.org/datasets/catalog/cifar100), [ImageNet](https://www.tensorflow.org/datasets/catalog/imagenet2012), [Jigsaw toxicity detection](https://www.tensorflow.org/datasets/catalog/wikipedia_toxicity_subtypes), etc), refer to the [Uncertainty Baselines](https://github.com/google/uncertainty-baselines) benchmark.

**Acknowledgement:** This notebook is based on the the [TensorFlow tutorial](https://www.tensorflow.org/tutorials/understanding/sngp) written by Jeremiah Liu.

## About SNGP

SNGP is a simple approach to improve a deep classifier's uncertainty quality while maintaining a similar level of accuracy and latency. Given a deep residual network, SNGP makes two simple changes to the model:

* It applies spectral normalization to the hidden residual layers.
* It replaces the Dense output layer with a Gaussian process layer.

>![SNGP](http://tensorflow.org/tutorials/understanding/images/sngp.png)


Compared to other uncertainty approaches (such as Monte Carlo dropout or Deep ensemble), SNGP has several advantages:

* It works for a wide range of state-of-the-art residual-based architectures (for example, (Wide) ResNet, DenseNet, or BERT).
* It is a single-model method—it does not rely on ensemble averaging. Therefore, SNGP has a similar level of latency as a single deterministic network, and can be scaled easily to large datasets like [ImageNet](https://github.com/google/uncertainty-baselines/tree/main/baselines/imagenet) and [Jigsaw Toxic Comments classification](https://github.com/google/uncertainty-baselines/tree/main/baselines/toxic_comments).
* It has strong out-of-domain detection performance due to the _distance-awareness_ property.

## Setup

**Select the `uncertainty_aware_models_kernel` for this Jupyter notebook.**

If the `uncertainty_aware_models_kernel` is not available, run the following cell which will create it:

In [ ]:
!cd ~/asl-ml-immersion && make uncertainty_aware_models_kernel

**Note:** you may need to restart the kernel to use updated packages.

In [ ]:
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import sklearn.datasets
import tensorflow as tf
from official.nlp.modeling.layers import (
    RandomFeatureGaussianProcess,
    SpectralNormalization,
    gaussian_process,
)

## The two moon dataset

In this notebook, we will demonstrate how to implement SNGP using a simple synthetic dataset,  the [scikit-learn two moon dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html). It consists of 2d points split into two classes, each of which has the shape of a crescent moon. 

Let us start by defining constants that control the visualization of the dataset:

In [ ]:
# Set the quality of the plots
plt.rcParams["figure.dpi"] = 140

# We will visualize the dataset in the following ranges
DEFAULT_X_RANGE = (-3.5, 3.5)
DEFAULT_Y_RANGE = (-2.5, 2.5)

# Defines the default colors
DEFAULT_CMAP = colors.ListedColormap(["#377eb8", "#ff7f00"])

# Define the number of points to evaluate in the default ranges
DEFAULT_N_GRID = 100

# Define the amount of injected noise in the dataset
NOISE = 0.1

The following function creates the training set. (The amount of noise in the data can be controlled by the noise argument.)

In [ ]:
def make_training_data(sample_size=500, noise=NOISE):
    """Create two moon training dataset."""
    train_examples, train_labels = sklearn.datasets.make_moons(
        n_samples=2 * sample_size, noise=noise
    )

    # Adjust data position slightly.
    train_examples[train_labels == 0] += [-0.1, 0.2]
    train_examples[train_labels == 1] += [0.1, -0.2]

    return train_examples, train_labels

Since we will be interested in understanding how our model behaves also outside of the data distribution support, the following function
generates a test dataset with points in a grid within `DEFAULT_X_RANGE` and `DEFAULT_Y_RANGE`, which we will use to visualize the confidence of our model within that grid.

In [ ]:
def make_testing_data(
    x_range=DEFAULT_X_RANGE, y_range=DEFAULT_Y_RANGE, n_grid=DEFAULT_N_GRID
):
    """Create a mesh grid in 2D space."""
    # testing data (mesh grid over data space)
    x = np.linspace(x_range[0], x_range[1], n_grid)
    y = np.linspace(y_range[0], y_range[1], n_grid)
    xv, yv = np.meshgrid(x, y)
    return np.stack([xv.flatten(), yv.flatten()], axis=-1)

To numerically evaluate our model uncertainty, we create an additional **out-of-domain (OOD)** of points grouped in a small cluster away from the training points. The model never observes these OOD examples during training.

In [ ]:
def make_ood_data(sample_size=500, means=(2.5, -1.75), variances=(0.01, 0.01)):
    return np.random.multivariate_normal(
        means, cov=np.diag(variances), size=sample_size
    )

Let us now visualize our training points (blue and orange) as well as our out-of-domain points (red):

In [ ]:
# Load the train, test and OOD datasets.
train_examples, train_labels = make_training_data(sample_size=500)
test_examples = make_testing_data()
ood_examples = make_ood_data(sample_size=500)

# Visualize
pos_examples = train_examples[train_labels == 0]
neg_examples = train_examples[train_labels == 1]

plt.figure(figsize=(7, 5.5))

plt.scatter(pos_examples[:, 0], pos_examples[:, 1], c="#377eb8", alpha=0.5)
plt.scatter(neg_examples[:, 0], neg_examples[:, 1], c="#ff7f00", alpha=0.5)
plt.scatter(ood_examples[:, 0], ood_examples[:, 1], c="red", alpha=0.1)

plt.legend(["Positive", "Negative", "Out-of-Domain"])

plt.ylim(DEFAULT_Y_RANGE)
plt.xlim(DEFAULT_X_RANGE)

plt.show()

In the plot above, the blue and orange represent the positive and negative classes, and the red represents the OOD data. 

> **Important:** A model that quantifies the uncertainty well is expected to be confident when close to training data  (i.e., $p(x_{test})$ close to 0 or 1), and be uncertain when far away from the training data (i.e., $p(x_{test})$ close to 0.5). Namely, our goal is to devise mechanisms preventing a model from beeing confidently wrong on input points that are very different from those seen during training. 

## The deterministic model

We start by a implementing a model with no special mechanism to deal with OOD data points, which we will take as our baseline. The architecture is a standard multi-layer residual network (ResNet) with dropout regularization.

### Exercise

Implement the `call` method so that it first passes the input through the `self.input_layer`, and then through ResNet blocks each formed by a layer in `self.dense_layers`, followed by a `Dropout` layer set with `dropout_rate`, and the addition of that ouput with the input of the block (addition which implements the ResNet skip connection). The method should then return the output of the last layer `self.classifier`.

In [ ]:
class DeepResNet(tf.keras.Model):
    """Defines a multi-layer residual network."""

    def __init__(
        self,
        num_classes,
        num_layers=3,
        num_hidden=128,
        dropout_rate=0.1,
        **classifier_kwargs,
    ):
        super().__init__()
        # Defines class meta data.
        self.num_hidden = num_hidden
        self.num_layers = num_layers
        self.dropout_rate = dropout_rate
        self.classifier_kwargs = classifier_kwargs

        # Defines the hidden layers.
        self.input_layer = tf.keras.layers.Dense(self.num_hidden)
        self.dense_layers = [self.make_dense_layer() for _ in range(num_layers)]

        # Defines the output layer.
        self.classifier = self.make_output_layer(num_classes)

    def call(self, inputs):
        # TODO
        pass

    def make_dense_layer(self):
        """Uses the Dense layer as the hidden layer."""
        return tf.keras.layers.Dense(self.num_hidden, activation="relu")

    def make_output_layer(self, num_classes):
        """Uses the Dense layer as the output layer."""
        return tf.keras.layers.Dense(num_classes, **self.classifier_kwargs)

Let us define our model with six hidden layers with 128 units each:

### Exercise

Instanciate in the cell below a `DeepResNet` model that 1) outputs the logit for a single class, 2) has 6 hidden layers with 128 neurons each.

In [ ]:
resnet_model = None  # TODO: Instanciate the network here

resnet_model.build((None, 2))
resnet_model.summary()

### Train model

The following cell defines the training configuration. Since the model outputs the logits for the classes and not the probabilities, we use the `SparseCategoricalCrossentropy` with `from_logits=True`:

### Exercise

In the cell below Compile the model with a `BinaryCrossentropy` loss, a `BinaryAccuracy` metrics, and the `Adam` optimizer.
Then train the model for 100 epochs with a batch size of 128.

In [2]:
# TODO: Compile and Train the DeepResNet model

### Visualize uncertainty

The following functions visualizes the model uncertainty by plotting uncertainty values (passed as the `test_uncertainty` array) of our model  at the 2D-grid test points stored in `test_examples`. (The function also takes a matplotlib `Axes` object in the parameter `ax` indicating where to display the plot as well as a the color palette to use in the `cmap` argument.)

In [ ]:
def plot_uncertainty_surface(test_uncertainty, ax, cmap=None):
    # Normalize uncertainty for better visualization.
    test_uncertainty = test_uncertainty / np.max(test_uncertainty)

    # Set view limits.
    ax.set_ylim(DEFAULT_Y_RANGE)
    ax.set_xlim(DEFAULT_X_RANGE)

    # Plot normalized uncertainty surface.
    pcm = ax.imshow(
        np.reshape(test_uncertainty, [DEFAULT_N_GRID, DEFAULT_N_GRID]),
        cmap=cmap,
        origin="lower",
        extent=DEFAULT_X_RANGE + DEFAULT_Y_RANGE,
        vmin=0,
        vmax=1,
        interpolation="bicubic",
        aspect="auto",
    )

    # Plot training data.
    ax.scatter(
        train_examples[:, 0],
        train_examples[:, 1],
        c=train_labels,
        cmap=DEFAULT_CMAP,
        alpha=0.5,
    )
    ax.scatter(ood_examples[:, 0], ood_examples[:, 1], c="red", alpha=0.1)

    return pcm

Now we can visualize the uncertainty in the predictions of our model in two ways. 


The first way is to plot the class probability values directly

$$p(x) = \textrm{sigmoid}(\textrm{logit}(x))$$

using the function `plot_uncertainty_surface`:

### Exercise

In the cell below compute the logits and the corresponding probabilities of the deep ResNet model evaluated on `test_examples`.

In [3]:
# TODO
resnet_logits = None
resnet_probs = None

A class probability value close to 0 or 1 at an input point indicates that the model is very confident in its predictions at this point, while a probability value close to  0.5 indicates that the model is uncertain. We see that our baseline model is very confident far away from the data, including on our small patch of red OOD points:

In [ ]:
_, ax = plt.subplots(figsize=(7, 5.5))

pcm = plot_uncertainty_surface(resnet_probs, ax=ax)

plt.colorbar(pcm, ax=ax)
plt.title("Class Probability, Deterministic Model")

plt.show()

In the plot above, the yellow and purple are the predictive probabilities for the two classes. The deterministic model did a good job in classifying the two known classes (blue and orange points) with a nonlinear decision boundary. However, it is not **distance-aware**, and classified the never-observed red out-of-domain (OOD) examples confidently as the orange class.


A second way to visualize the model uncertainty is by computing the [predictive variance](https://en.wikipedia.org/wiki/Bernoulli_distribution#Variance) of our model predictions:

$$\textrm{var}(x) = p(x) * (1 - p(x))$$

A higher variance indicates predictions that are more uncertain, while a lower variance indicates more confident predictions. Let us compute the predictive variance and visualize it on our 2D grid of `test_examples` points:

### Exercise

In the cell below, compute the predictive variance of the probability of the deep ResNet model evaluated on `test_examples`.

In [ ]:
# TODO
resnet_uncertainty = None

Let us now plot the predictive variance (note that the `plot_uncertainty_surface` function scales the values to be in the interval $[0,1]$):

In [ ]:
_, ax = plt.subplots(figsize=(7, 5.5))

pcm = plot_uncertainty_surface(resnet_uncertainty, ax=ax)

plt.colorbar(pcm, ax=ax)
plt.title("Predictive Uncertainty, Deterministic Model")

plt.show()

In the plot above, the yellow indicates high uncertainty, and the purple indicates low uncertainty. A deterministic ResNet's uncertainty depends only on the test examples' distance from the decision boundary. This leads the model to be over-confident when out of the training domain. The next section shows how SNGP behaves differently on this dataset.

## The SNGP model

Our goal next is to tweak the deterministic model to make it uncertainty aware using SNGP, which relies on two components, `SpectralNormalization` and `RandomFeatureGaussianProcess`. Both components are available at the tensorflow_model's [built-in  layers](https://github.com/tensorflow/models/tree/master/official/nlp/modeling/layers). We can import these layers along with utilities for Gaussian processes using the following import:

```python
from official.nlp.modeling.layers import (
    RandomFeatureGaussianProcess,
    SpectralNormalization,
    gaussian_process,
)
```

Implementing SNGP on the top of our deterministic model requires two steps:

1) Regularize the hidden layers of our network with spectral normalization to ensure that the distance between two input points $x$ and $x'$ is roughly preserved by the hidden layers $h$ (see [Sec. 3.2 here](https://arxiv.org/pdf/2006.10108.pdf) for more details)
2) Replace the deterministics dense layer of our network that ouputs the logits with a probabilistic layer that outputs a Gaussian distribution for the logits instead. The variance of this distribution will help us quantify the uncertainty in our model predictions: Higher variance corresponding to higher uncertainty. 

>![SNGP](http://tensorflow.org/tutorials/understanding/images/sngp.png)


Let's inspect these two components in more detail. (You can also jump to [the full SNGP model](#full-sngp-model) section to learn how SNGP is implemented.)

#### 1) First SNGP component: The `SpectralNormalization` (SN) layer

[`SpectralNormalization`](https://github.com/tensorflow/models/blob/master/official/nlp/modeling/layers/spectral_normalization.py) is a Keras layer wrapper. It can be applied to an existing Dense layer like this:

```python
dense = tf.keras.layers.Dense(units=10)
dense = SpectralNormalization(dense, norm_multiplier=0.9)
```

Spectral normalization regularizes the hidden weight $W$ matrix by gradually guiding its spectral norm (that is, the largest eigenvalue of $W$) toward the target value `norm_multiplier`. 

Intutitively, the spectral norm of a dense-layer weight-matrix  controls how much the distance between input points is strecthed or shrinked by the layer. Because of that, constraining the spectral norm of each of the dense layers to be close to 1 encourages the network hidden representation $h(x)$ to roughly maintain the distance between the original input points $x$. 

This way the Gaussian Proccess layer (which we describe next and which takes its input only from the hidden representation $h(x)$) will have knowledge of the original distance between the input points, which it will transform into a measure of incertainty. (See [Sec. 3.2 here](https://arxiv.org/pdf/2006.10108.pdf) for more details)


**Note:** Usually it is preferable to set `norm_multiplier` to a value smaller than 1. However in practice, it can be also relaxed to a larger value to ensure the deep network has enough expressive power.

#### 2) Second SNGP component: The Gaussian Process (GP) layer

In our baseline deterministic network, a number of ResNet layers create a hidden representation $h$ or embeddings for the input points:

```python
embeddings = h(x_batch)
```

Then we pass these embeddings to a dense layer to create the logits (used by a sigmoid layer for classification):

```python
logits = Dense(1)(embeddings)
```

As we saw these logits are generally not distance-aware, since they their absolute values can be high far away from the data distribution, yielding very confident probabilities close to 0 or 1 when applying the final sigmoid layer:

```python
probs = tf.nn.sigmoid(logits)
```

Our goal is to scale the logits by a quantity that is large far away from the data, so that the scaled logits are close to zero and the probability prediction based on the scaled logits are more uncertain (i.e, closer to 0.5). We also want this scaling to be close to 1 in the regions of the input space where we have data points, so that the scaling doesn't impact the quality of the classifier in regions where the data exists. 

How can we create such a scaling? That's where the Gaussian Process layer comes to the rescue. Instead of applying the dense layer `Dense(1)` to the embeddings, we will replace it by a Gaussian Proccess layer:

```python
sngp_logits, covmat = RandomFeatureGaussianProcess(1)(embeddings)
```

(Don't worry about the `RandomFeature` part, which is just a particular way to approximate the Gaussion Process; see [this paper](https://people.eecs.berkeley.edu/~brecht/papers/07.rah.rec.nips.pdf) for details if you are interested.)

The GP layer still outputs a single logit for each data point in `x_batch` similarly to the dense layer `Dense(1)`, but now it also outputs a **covariance matrix** `covmat` of shape `(batch_size, batch_size)` on the top of the logits. 


Basically, the GP layer outputs the mean vector of a Gaussian distribution (i.e. the vector of logits) along with its covariance matrix. In particular, the diagonal elements of this covariance matrix correspond to the  variance of each logit prediction. We can retrieve these diagonal element as follows: 

```python
sngp_variance = tf.linalg.diag_part(sngp_covmat)[:, None]
```

Typically, a Gaussian Process will assign a very low variance (close to zero) to points in the test data that are close to the training points, while the variance will be higher for points that are far away from the training point. We can now use this property to scale our logits:

$$\frac{\textrm{logit}(x)}{\sqrt{1+ \lambda * \sigma^2(x)}}$$

where $\sigma^2(x)$ is the SNGP variance, and $\lambda$ is an hyper-parameter (often chosen as $\pi/8$ or $3/\pi^2$).
In code, this translate into

```python

sngp_logits_adjusted = sngp_logits / tf.sqrt(1.0 + (np.pi / 8.0) * sngp_variance)
```

Observe that the scaling will be close to 1 in regions where the variance is close to zero (i.e. in regions where we have training points), while it will be large outside of that regions when the GP process variance is high. 

**Note:** See the [original tutorial](https://www.tensorflow.org/tutorials/understanding/sngp) for an explanation on how to modify the GP process to the case of several classes.

Let's now see how to tweak the deterministic model we have been considering so far to implement SNGP on the top of it.

<a name="full-sngp-model"></a>
#### Implementation of the full SNGP model

Given the base class `DeepResNet`, the SNGP model can be implemented easily by  modifying the residual network's hidden and output layers. For compatibility with Keras `model.fit()` API, also modify the model's `call()` method so it only outputs `logits` during training.

### Exercise

In the cell below:
1) Implement `make_dense_layer` so that it applies `SpectralNormalization` to the parent class dense layer returned by the method of the same name in the parent class (**Hint:** Use `super().make_dense_layer()` in the derived class method to access the original dense layer). Set the `norm_multiplier` argument of the `SpectralNormalization` layer to `self.spec_norm_bound`.
1) Implement `make_output_layer` so that it returns a `RandomFeatureGaussianProcess` layer instanciated with `num_classes` and all the possible additional arguments in `**self.classifier_kwarg`
1) Implement the `call` method so that it returns the logits along with the covariance matrix when `training=False` and `return_covmat=False` and only the logits otherwise

In [ ]:
class DeepResNetSNGP(DeepResNet):
    def __init__(self, spec_norm_bound=0.9, **kwargs):
        self.spec_norm_bound = spec_norm_bound
        super().__init__(**kwargs)

    def make_dense_layer(self):
        """Applies spectral normalization to the hidden layer."""
        pass  # TODO

    def make_output_layer(self, num_classes):
        """Uses Gaussian process as the output layer."""
        pass  # TODO

    def call(self, inputs, training=False, return_covmat=False):
        pass  # TODO

<a name="covariance-reset-callback"></a>

For technical reasons, the GP layer needs to reset the covariance matrix at the beginning of each new epoch so that the data points are not counted several times during training. For that we can create a Keras callback by subclassing from `tf.keras.callbacks.Callback` and by implementing the 
`on_epoch_begin` method, which will be then invoked before a new training epoch begins. Reseting the GP layer will be easy since we exposed the last layer directly through the `classifier` attribute in the `DeepResNet` parent class:

In [ ]:
class ResetCovarianceCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        """Resets covariance matrix at the beginning of the epoch."""
        if epoch > 0:
            self.model.classifier.reset_covariance_matrix()

Now that we have this callback, we could add it to the `callbacks` argument to the `fit` method when lauching the training. Because this callback is not optional and must be called everytime we train with the GP layer, this could be error prone. To avoid this, we will instead include the callback it directly by default to the `fit` method of the model itself. This way we will be sure that it is always called:

### Exercise

Implement the `fit` method in the cell below so that it adds the `ResetCovarianceCallback()` callback automatically to the list of the callbacks passed to the method through the `**kwargs` argument before calling the parent class `fit` method. 

**Hint:** You can call the parent `fit` method using `super().fit(*args, **kwargs)` and you can access the callback list using `list(kwargs.get("callbacks", []))`.

In [ ]:
class DeepResNetSNGPWithCovReset(DeepResNetSNGP):
    def fit(self, *args, **kwargs):
        pass  # TODO

### Train the model

Now we are ready to train the SGNP model!

In [ ]:
resnet_config

In [ ]:
sngp_model = DeepResNetSNGPWithCovReset(**resnet_config)
sngp_model.compile(**train_config)
sngp_model.fit(train_examples, train_labels, **fit_config)

### Scale the logits

First we need to compute the logits and the covariance matrix using the SNGP model:

In [ ]:
sngp_logits, sngp_covmat = sngp_model(test_examples, return_covmat=True)

This gives us the vector of means of the Gaussian (which are our logits), as well as the Gaussian probability covariance matrix:

In [ ]:
sngp_logits.shape

In [ ]:
sngp_covmat.shape

The diagonal entries of `sngp_covmat` correspond to the variances of the logit vector components, which we collect into a single vector: 

In [ ]:
sngp_variance = tf.linalg.diag_part(sngp_covmat)[:, None]

We can now rescale the logits using the SNGP variances according to the following formula, which will produce logits close to zero when the variance is high:

$$\frac{\textrm{logit}(x)}{\sqrt{1+ \lambda * \sigma^2(x)}}$$

where $\sigma^2(x)$ is the SNGP variance, and $\lambda$ is often chosen as $\pi/8$ or $3/\pi^2$. Note that instead of fixing $\lambda$ to a fixed value, you can also treat it as a hyperparameter, and tune it to optimize the model's calibration performance. This is known as [temperature scaling](http://proceedings.mlr.press/v70/guo17a.html) in the deep learning uncertainty literature. (For details regarding the scaling formula see the [mean-field method](https://arxiv.org/abs/2006.07584).)

In code, this gives:

In [ ]:
logits_adjusted = sngp_logits / tf.sqrt(1.0 + (np.pi / 8.0) * sngp_variance)

However the `gaussian_process` module provides the function `gaussian_process.mean_field_logits` that does this scaling for us. Let us package the scaling and the application of the sigmoid to the logits into a single function:

In [ ]:
def compute_sngp_probability(logits, covmat, lambda_param=np.pi / 8.0):
    # Computes uncertainty-adjusted logits using the built-in method.
    logits_adjusted = gaussian_process.mean_field_logits(
        logits, covmat, mean_field_factor=lambda_param
    )

    return tf.nn.sigmoid(logits_adjusted)

In [ ]:
sngp_logits, sngp_covmat = sngp_model(test_examples, return_covmat=True)
sngp_probs = compute_sngp_probability(sngp_logits, sngp_covmat)

You can now put everything together. The entire procedure—training, evaluation and uncertainty computation—can be done in just five lines:

### Exercise

In the cell below, assemble all what we did into a single function `train_and_test_sngp` below to that it instanciates a `DeepResNetSNGPWithCovReset`, compiles it, and trains it on `train_examples`. Then it should retrieve the logits as well as the covariance matrix evaluated on `test_examples` before returned the rescaled probabilities using the `compute_sngp_probability` function

In [ ]:
def train_and_test_sngp(train_examples, test_examples):
    # TODO
    return sngp_probs

In [ ]:
sngp_probs = train_and_test_sngp(train_examples, test_examples)

### SNGP Summary

The following function display the class probability $p$ (on the left) and the predictive uncertainty $p(1-p)$ (on the right) of the SNGP model.

In [ ]:
def plot_predictions(pred_probs, model_name=""):
    """Plot normalized class probabilities and predictive uncertainties."""
    # Compute predictive uncertainty.
    uncertainty = pred_probs * (1.0 - pred_probs)

    # Initialize the plot axes.
    fig, axs = plt.subplots(1, 2, figsize=(14, 5))

    # Plots the class probability.
    pcm_0 = plot_uncertainty_surface(pred_probs, ax=axs[0])
    # Plots the predictive uncertainty.
    pcm_1 = plot_uncertainty_surface(uncertainty, ax=axs[1])

    # Adds color bars and titles.
    fig.colorbar(pcm_0, ax=axs[0])
    fig.colorbar(pcm_1, ax=axs[1])

    axs[0].set_title(f"Class Probability, {model_name}")
    axs[1].set_title(f"(Normalized) Predictive Uncertainty, {model_name}")

    plt.show()

In [ ]:
plot_predictions(sngp_probs, model_name="SNGP")

Remember that in the class probability plot (left), the yellow and purple are class probabilities. When close to the training data domain, SNGP correctly classifies the examples with high confidence (i.e., assigning near 0 or 1 probability). When far away from the training data, SNGP gradually becomes less confident, and its predictive probability becomes close to 0.5 while the (normalized) model uncertainty rises to 1.

Compare this to the uncertainty surface of the deterministic model: 

In [ ]:
plot_predictions(resnet_probs, model_name="Deterministic")

As mentioned earlier, a deterministic model is not _distance-aware_. Its uncertainty is defined by the distance of the test example from the decision boundary. This leads the model to produce overconfident predictions for the out-of-domain examples (red).

## Comparison with other uncertainty approaches (Optional)

This section compares the uncertainty of SNGP with [Monte Carlo dropout](https://arxiv.org/abs/1506.02142) and [Deep ensemble](https://arxiv.org/abs/1612.01474).

Both of these methods are based on Monte Carlo averaging of multiple forward passes of deterministic models. First, set the ensemble size $M$.

In [ ]:
num_ensemble = 10

### Monte Carlo dropout

Given a trained neural network with Dropout layers, Monte Carlo dropout computes the mean predictive probability

$$E(p(x)) = \frac{1}{M}\sum_{m=1}^M \textrm{softmax}(\textrm{logit_m}(x))$$

by averaging over multiple Dropout-enabled forward passes $\{\textrm{logit_m}(x)\}_{m=1}^M$.

In [ ]:
def mc_dropout_sampling(test_examples):
    # Enable dropout during inference.
    return resnet_model(test_examples, training=True)

In [ ]:
# Monte Carlo dropout inference.
dropout_logit_samples = [
    mc_dropout_sampling(test_examples) for _ in range(num_ensemble)
]
dropout_prob_samples = [
    tf.nn.sigmoid(dropout_logits) for dropout_logits in dropout_logit_samples
]
dropout_probs = tf.reduce_mean(dropout_prob_samples, axis=0)

In [ ]:
plot_predictions(dropout_probs, model_name="MC Dropout")

### Deep ensemble

Deep ensemble is a state-of-the-art (but expensive) method for deep learning uncertainty. To train a Deep ensemble, first train $M$ ensemble members.

In [ ]:
# Deep ensemble training
resnet_ensemble = []
for _ in range(num_ensemble):
    resnet_model = DeepResNet(**resnet_config)
    resnet_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    resnet_model.fit(train_examples, train_labels, verbose=0, **fit_config)

    resnet_ensemble.append(resnet_model)

Collect logits and compute the mean predictive probability $E(p(x)) = \frac{1}{M}\sum_{m=1}^M \textrm{sigmoid}(\textrm{logit_m}(x))$.

In [ ]:
# Deep ensemble inference
ensemble_logit_samples = [model(test_examples) for model in resnet_ensemble]
ensemble_prob_samples = [
    # tf.nn.softmax(logits, axis=-1)[:, 0] for logits in ensemble_logit_samples
    tf.nn.sigmoid(logits)
    for logits in ensemble_logit_samples
]
ensemble_probs = tf.reduce_mean(ensemble_prob_samples, axis=0)

In [ ]:
plot_predictions(ensemble_probs, model_name="Deep ensemble")

Both the Monte Carlo Dropout and Deep ensemble methods improve the model's uncertainty ability by making the decision boundary less certain. However, they both inherit the deterministic deep network's limitation in lacking distance awareness.

## Resources and further reading

* Check out the [SNGP-BERT tutorial](https://www.tensorflow.org/text/tutorials/uncertainty_quantification_with_sngp_bert) for an example of applying SNGP on a BERT model for uncertainty-aware natural language understanding.
* Go to the [Uncertainty Baselines GitHub repo](https://github.com/google/uncertainty-baselines) for the implementation of SNGP model (and many other uncertainty methods) on a wide variety of benchmark datasets (for example, [CIFAR](https://www.tensorflow.org/datasets/catalog/cifar100), [ImageNet](https://www.tensorflow.org/datasets/catalog/imagenet2012), [Jigsaw toxicity detection](https://www.tensorflow.org/datasets/catalog/wikipedia_toxicity_subtypes), etc).
* For a deeper understanding of the SNGP method, check out the paper titled [Simple and Principled Uncertainty Estimation with Deterministic Deep Learning via Distance Awareness](https://arxiv.org/abs/2006.10108).


Copyright 2022 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.